In [8]:
import mdtraj as md
from ase import Atoms
import nglview as nv
import networkx as nx

import torch
import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, SAGPooling, InnerProductDecoder
import torch_geometric.data as data
from torch_geometric.utils.convert import to_networkx

import numpy as np
import matplotlib.pyplot as plt
from PyEMD import EMD
import emd


In [9]:
emd=EMD()

In [10]:
from PyEMD import EMD
import emd
emd=EMD()

In [11]:
import torch
import torch.nn as nn

# import _emd_ext._emd as emd


class EMDFunction(torch.autograd.Function):
	@staticmethod
	def forward(self, xyz1, xyz2):
		cost, match = emd.emd_forward(xyz1, xyz2)
		self.save_for_backward(xyz1, xyz2, match)
		return cost


	@staticmethod
	def backward(self, grad_output):
		xyz1, xyz2, match = self.saved_tensors
		grad_xyz1, grad_xyz2 = emd.emd_backward(xyz1, xyz2, match)
		return grad_xyz1, grad_xyz2




class EMDLoss(nn.Module):
	'''
	Computes the (approximate) Earth Mover's Distance between two point sets. 

	IMPLEMENTATION LIMITATIONS:
	- Double tensors must have <=11 dimensions
	- Float tensors must have <=23 dimensions
	This is due to the use of CUDA shared memory in the computation. This shared memory is limited by the hardware to 48kB.
	'''

	def __init__(self):
		super(EMDLoss, self).__init__()

	def forward(self, xyz1, xyz2):

		assert xyz1.shape[-1] == xyz2.shape[-1], 'Both point sets must have the same dimensionality'
		if xyz1.dtype == torch.float64 and xyz1.shape[-1] > 11:
			error('Tensors of type double can have a maximum of 11 dimensions')
		if xyz1.dtype == torch.float32 and xyz1.shape[-1] > 23:
			error('Tensors of type float can have a maximum of 23 dimensions')

		return EMDFunction.apply(xyz1, xyz2)

In [12]:
dist=EMDLoss()
p1 = torch.rand(1,5,3).cuda().double()
p2 = torch.rand(1,10,3).cuda().double()
p1.requires_grad = True
p2.requires_grad = True

cost = dist(p1, p2)


print(cost)
loss = torch.sum(cost)
print(loss)
loss.backward()
print(p1.grad)
print(p2.grad)


AttributeError: 'EMD' object has no attribute 'emd_forward'